In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# PCA
## Load data

In [ ]:
X = pd.read_csv('./leaf.csv', header=None).values
X.shape

In [ ]:
# scale data
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
X_scaled = (X-mean) / std

# calculate covariance matrix
cov = np.cov(X_scaled.T)

# calculate eigenvectors and eigenvalues
eig_vals, eig_vecs = np.linalg.eig(cov)
plt.plot(np.sort(eig_vals))
plt.xlim([0,15])
plt.xlabel('eigenvalue index')
plt.ylabel('value')